# Morphology (Tutorial 3)
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Exercise 1 - Connected Components](#Exercise-1---Connected-Components)
3.   [Exercise 2 - Dilation](#Exercise-2---Dilation)
4.   [Exercise 3 - Erosion](#Exercise-3---Erosion)
5.   [Exercise 4 - Opening](#Exercise-4---Opening)
6.   [Exercise 5 - Closing](#Exercise-5---Closing)
7.   [Exercise 6 - Segmentation](#Exercise-6---Segmentation)

# Imports

Only 4 libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms
* os - File traversal
* tqdm.notebook - tqdm progress bars, but for ipynb files
* Classes - Custom classes written by me for this assignment

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm

# A slightly modified generateHistograms function from tutorial 1

def generateHistogram(image, title, detail):
    hist = cv2.calcHist(image, [0], None, [255], [0, 255])
    plt.ylabel('Pixels')
    plt.xlabel('Intensity')
    plt.title(title)
    plt.plot(hist)
    plt.xlim([0, 256])
    plt.savefig(detail + '/'+ title + ' histogram.png')
    plt.clf()

